In [ ]:
import json

with open('KIRO-medium.json') as f:
    data = json.load(f)


In [ ]:
print(data.keys())
for key in data.keys():
    print(key+" : "+str(len(data[key])))

dict_keys(['land_substation_cable_types', 'wind_turbines', 'wind_scenarios', 'substation_locations', 'substation_types', 'general_parameters', 'substation_substation_cable_types'])
land_substation_cable_types : 24
wind_turbines : 135
wind_scenarios : 101
substation_locations : 30
substation_types : 24
general_parameters : 7
substation_substation_cable_types : 21


In [ ]:
data['general_parameters']

{'fixed_cost_cable': 5.0,
 'variable_cost_cable': 0.5,
 'curtailing_penalty': 4500.0,
 'curtailing_cost': 45.0,
 'main_land_station': {'x': 0, 'y': 0},
 'maximum_power': 18.0,
 'maximum_curtailing': 500.0}

In [ ]:
import json
import numpy as np
def parse_input_data(input_file):
    with open(input_file) as f:
        data = json.load(f)
    return data

In [ ]:
def calculate_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

def calculate_distance_matrix(points):
    distance_matrix = np.zeros((len(points), len(points)))
    for i in range(len(points)):
        for j in range(len(points)):
            distance_matrix[i][j] = calculate_distance(points[i], points[j])
    return distance_matrix

def calculate_distance(idx1, idx2, distance_matrix):
    return distance_matrix[idx1,idx2]

def calculate_cable_cost_ls(cable_type, length):
    return data['land_substation_cable_types'][cable_type]['variable_cost'] * length + data['land_substation_cable_types'][cable_type]['fixed_cost']

def calculate_cable_cost_ss(cable_type, length):
    return data['substation_substation_cable_types'][cable_type]['variable_cost'] * length + data['substation_substation_cable_types'][cable_type]['fixed_cost']

In [ ]:
input_file = 'KIRO-medium.json'
print(data.keys())

dict_keys(['land_substation_cable_types', 'wind_turbines', 'wind_scenarios', 'substation_locations', 'substation_types', 'general_parameters', 'substation_substation_cable_types'])


In [ ]:
class problem():
    def __init__(self,input_file):
        self.data = parse_input_data(input_file)
        self.N_turbines = len(self.get('wind_turbines'))
        self.N_substations = len(self.get('substation_locations'))
        self.distance_matrix_ss = self.get_distance_matrix_ss()
        self.distance_matrix_st = self.get_distance_matrix_st()
        self.distance_matrix_ls = self.get_distance_matrix_ls()

        self.count=0 # à supprimer

    def get_distance_matrix_st(self):
        # Compute the distance matrix between substations and turbines
        distance_matrix_st = np.zeros((self.N_substations, self.N_turbines))
        for i in range(self.N_substations):
            position_i = self.get('substation_locations')[i]
            for j in range(self.N_turbines):
                position_j = self.get('wind_turbines')[j]
                distance_matrix_st[i][j] = np.sqrt((position_i["x"]-position_j["x"])**2 + (position_i["y"]-position_j["y"])**2)
        return distance_matrix_st

    def get_distance_matrix_ss(self):
        #Compute the distance matrix between substations
        distance_matrix_ss = np.zeros((self.N_substations, self.N_substations))
        for i in range(self.N_substations):
            position_i = self.get('substation_locations')[i]

            for j in range(self.N_substations):
                position_j = self.get('substation_locations')[j]
                distance_matrix_ss[i][j] = np.sqrt((position_i["x"]-position_j["x"])**2 + (position_i["y"]-position_j["y"])**2)
        return distance_matrix_ss

    def get_distance_matrix_ls(self):
        #COmpute the distance matrix between land station and substations
        distance_matrix_ls = np.zeros((self.N_substations))
        position_land = self.get('general_parameters')['main_land_station']
        for i in range(self.N_substations):
            position_i = self.get('substation_locations')[i]
            distance_matrix_ls[i] = np.sqrt((position_i["x"]-position_land["x"])**2 + (position_i["y"]-position_land["y"])**2)
        return distance_matrix_ls

    def json_overview(self):
        print("TOTAL NUMBERS")
        for key in data.keys():
            print("\t"+key+" : "+str(len(data[key])))
    def get(self,key,view=False):
        if view:
            print(data[key])
        return data[key]



Ess is not initialised
the calculus are not good



In [ ]:
import numpy as np
import random as rd
from itertools import product
from copy import deepcopy

class solution():
    def __init__(self,problem_instance):
        self.problem_instance = problem_instance
        self.V_substation = [] #(id_location, station_type,land_cable_type,id_backup_station) # Id of locations start from 1 and 0 represents either the ground station or a dummy value
        self.V_turbines = [] #(id_turbine, id_location) it's the location of the station with whom it's liked
        self.E_ss = [] # (id1, id2, cable_type, length)
        self.compute_table = []


    def random_heuristic_construction(self):
        #we only build a random substations :

        j=rd.randint(self.problem_instance.N_substations//2,self.problem_instance.N_substations -1)

        chosen_substations_index = rd.sample(range(self.problem_instance.N_substations), k=j)


        # def find_nearest_center(point_index, centers_indexes):
        #   #convert them to points
        #   data_substations=self.problem_instance.get('substation_locations')
        #   centers=[]
        #   for i in  centers_indexes :
        #     centers.append(np.array([data_substations[i]["x"],data_substations[i]["y"]]))
        #   centers=np.array(centers)


        #   #points :
        #   data=self.problem_instance.get('wind_turbines')
        #   point=np.array([data[point_index]["x"],data[point_index]["y"]])

        #   distances = np.linalg.norm(centers - point, axis=1)
        #   index=np.argmin(distances)

        #   return centers_indexes[index]



        for i in range(self.problem_instance.N_turbines):
            #in our terms
            substation_index=chosen_substations_index[i%j]

            self.V_turbines.append(np.array([i+1,substation_index+1]))

        for substation_index in chosen_substations_index:
            #add it to V_substations
            station_type=  rd.randint(0, len(self.problem_instance.data["substation_types"])-1)
            land_cable_type=  rd.randint(0, len(self.problem_instance.data["land_substation_cable_types"])-1) # Not used for the moment
            self.V_substation.append(np.array([substation_index+1,station_type,0,None]))

        p=j//2

        chosen_substations = rd.sample(range(j), k=j) #2 random elements



        for i in range(p): #link it with the not used neighbor
              # center_indexes=[u for u in range(j) if u != i ]
              # l=find_nearest_center(i, center_indexes)

              index=chosen_substations_index[i]
              index_to_link=chosen_substations_index[i+p]
              cable_type_index = rd.randint(0, len(self.problem_instance.data["substation_substation_cable_types"])-1)
              self.E_ss.append(np.array([index+1, index_to_link+1 ,cable_type_index ,0]))





              for v_sub in self.V_substation :
                  if v_sub[0]==index+1:
                      v_sub[3]==index_to_link+1  #Pas None


    #was not used


    def get_compute_table(self):
        # For each station, we get number of turbines linked to it, rating of cable to land, rating of substation, backup_station(defaut = None)
        # Number of turbines linked to , Rating, Rating, Backup_ID, probablity of failure

        compute_table = np.zeros((self.problem_instance.N_substations+1,5))
        land_substation_cable_types = self.problem_instance.get('land_substation_cable_types')
        substation_types = self.problem_instance.get('substation_types')
        for id_turbine, id_location in self.V_turbines:
            compute_table[id_location][0] += 1
        for id_location,station_type, land_cable_type,id_backup in self.V_substation:
          if station_type < len(substation_types) :
            compute_table[id_location][1] = land_substation_cable_types[land_cable_type]['rating']
            compute_table[id_location][2] = substation_types[station_type]['rating']
            compute_table[id_location][4] = substation_types[station_type]['probability_of_failure']+land_substation_cable_types[land_cable_type]['probability_of_failure'] #if it's activated the sum is equal to one
        # visited_locations=[]
        for id_location, id_backup, sub_sub_cable_type , l in self.E_ss :
            compute_table[id_location][3] = land_substation_cable_types[sub_sub_cable_type]['rating']
            # visited_locations.append(id_location)


        #to have an error , it's on purpose :
        # for id in range(self.problem_instance.N_substations+1):
        #   if id not in visited_locations :
        #       compute_table[id_location][3]=  None




        return compute_table

    def normalize_solution(self):
      for v_sub in self.V_substation :
        v_sub[1]+=1
        v_sub[2]+=1
      for e in self.E_ss :
        e[2]+=1





    def get_operational_cost_per_scenario(self,power_generation):

        # Compute production cost in case of no failure
        cables_substation_types_rating = self.problem_instance.get("substation_substation_cable_types")

        production_cost = self.compute_table[:,0]*power_generation-np.minimum(self.compute_table[:,1],self.compute_table[:,2])



        # Compute no failure cost
        no_failure_cost_per_station = np.maximum(production_cost,0)
        # print("  no_failure_cost_per_station ",   no_failure_cost_per_station )
        no_failure_cost = np.sum(no_failure_cost_per_station) #>= 0



        ######################################"
        curtailing_penalty =self.problem_instance.get("general_parameters")["curtailing_penalty"]
        curtailing_cost =self.problem_instance.get("general_parameters")["curtailing_cost"]
        maximum_curtailing = self.problem_instance.get("general_parameters")["maximum_curtailing"]

         #We have to apply  c(C) here
        total_cost_no_failure=np.maximum(no_failure_cost-maximum_curtailing,0)*curtailing_penalty + curtailing_cost*no_failure_cost # >0
        # print(" total_cost_no_failure",  total_cost_no_failure)




        # Initialize cost_per_failure_table that contains the cost in case(or not) of failure of each station and the corresponding probability
        cost_per_failure = np.zeros(self.problem_instance.N_substations+1)
        cumulative_probability = 0   #I changed this
        # Compute failure cost of each station
        for id_location,station_type, land_cable_type,id_backup in self.V_substation:
            # Get the probability of failure, insert it to cost_per_failure_table and add it to cumulative_probability at index 0
            cumulative_probability += self.compute_table[id_location,4]
            # Compute the production of the failed station
            production_of_failed_station = self.compute_table[id_location,0]*power_generation
            # If it has a backup station
            if id_backup != None:
                    lost_energy = production_of_failed_station-self.compute_table[id_location,3]
                    # Compute the energy received by the backup station
                    cost_per_failure[id_location] = np.max(lost_energy,0) + max(production_cost[id_backup]+self.compute_table[id_location,3]+ min(lost_energy,0),0)

                    # if(self.problem_instance.count==1):
                    #     self.problem_instance.count+=1
                    #     print("Error --------------------------------------------------------------------")

            else:
                # If it has no backup station, the cost is the production of the failed station
                cost_per_failure[id_location] = no_failure_cost_per_station[id_location]

        final_cost_failure = (np.maximum(cost_per_failure-maximum_curtailing,0)*curtailing_penalty + curtailing_cost*cost_per_failure)
        # print("final_cost_failure" , final_cost_failure)
        total_cost = np.sum(self.compute_table[:,4]*(final_cost_failure)) +total_cost_no_failure*(1- cumulative_probability)
        # print("total_cost" ,total_cost)
        return total_cost


    def get_operational_cost(self):
        self.compute_table = self.get_compute_table()
        scenarios = self.problem_instance.get('wind_scenarios')
        cumulative_cost = 0
        for scenario in scenarios:
            cumulative_cost += scenario['probability']*self.get_operational_cost_per_scenario(scenario["power_generation"])
        # print("cumulative cost" ,cumulative_cost)
        return cumulative_cost

    def get_construction_cost(self):
        # Compute Construction cost
        # Output is a dictionary with the following keys
        # substation_substation_cost
        # substation_turbine_cost
        # substation_cost
        # land_substation_cost
        construction = dict()

        construction_cost = 0
        substation_substation_cable_types = self.problem_instance.get('substation_substation_cable_types')
        for id1,id2,cable_type,length in self.E_ss:
              construction_cost += self.problem_instance.distance_matrix_ss[id1-1][id2-1]*substation_substation_cable_types[cable_type]['variable_cost']+substation_substation_cable_types[cable_type]['fixed_cost']

        construction.__setitem__('substation_substation_cost',construction_cost)
        # Compute Construction cost of cables between substations and turbines
        # Formulae is length*variable_cost + fixed_cost

        construction_cost = 0
        general_parameters = self.problem_instance.get('general_parameters')
        for id_turbine, id_location in self.V_turbines:
            construction_cost += self.problem_instance.distance_matrix_st[id_location-1][id_turbine-1]*general_parameters['variable_cost_cable']+general_parameters['fixed_cost_cable']
        construction.__setitem__('substation_turbine_cost',construction_cost)

        # Compute Construction cost of substations

        construction_cost_ = 0
        substation_types = self.problem_instance.get('substation_types')
        for id_location,station_type, land_cable_type,id_backup in self.V_substation:
          if station_type < len(substation_types):
            construction_cost_ += substation_types[station_type]['cost']
        construction.__setitem__('substation_cost',construction_cost_)

        # Compute Construction cost of cables between substations and land
        # Formulae is length*variable_cost + fixed_cost
        land_substation_cable_types = self.problem_instance.get('land_substation_cable_types')
        construction_cost__ = 0
        for id_location,station_type, land_cable_type,id_backup in self.V_substation:
              construction_cost__ += self.problem_instance.distance_matrix_ls[id_location-1]*land_substation_cable_types[land_cable_type]['variable_cost']+land_substation_cable_types[land_cable_type]['fixed_cost']
        construction.__setitem__('land_substation_cost',construction_cost__)
        return construction

    def get_cost(self):
            cost = 0
            construction_cost = self.get_construction_cost()
            for key in construction_cost.keys():
                cost += construction_cost[key]
            cost_final=cost + self.get_operational_cost()
            return cost_final
    def solution_project_format(self):
        solution = dict()
        L = []
        for e in self.V_substation:
            substations = {"id": e[0], "land_cable_type": e[2], "substation_type": e[1]}
            L.append(substations)
        L.sort(key=lambda x: x['id'])
        solution["substations"] = L

        L = []
        for e in self.E_ss:
            substation_substation_cables = {"substation_id": e[0], "other_substation_id": e[1], "cable_type": e[2]}
            L.append(substation_substation_cables)
        L.sort(key=lambda x: x['substation_id'])
        solution["substation_substation_cables"] = L

        L = []
        for e in self.V_turbines:
            turbines = {"id": e[0], "substation_id": e[1]}
            L.append(turbines)
        L.sort(key=lambda x: x['id'])
        solution["turbines"] = L

        return solution






In [ ]:
from typing_extensions import ParamSpecArgs
from os import rmdir
from copy import deepcopy
class SimulatedAnnealing():
    def __init__(self, problem_instance, initial_solution, temperature=1000, cooling_rate=0.2,number_iteration=100):
        self.problem_instance = problem_instance
        self.current_solution = initial_solution
        self.temperature = temperature
        self.cooling_rate = cooling_rate
        self.best_solution = initial_solution
        self.number_iteration=number_iteration

    def acceptance_probability(self, new_cost, current_cost, temperature):
        if new_cost < current_cost:
            return 1.0
        return np.exp((current_cost - new_cost) / temperature)


    def _choose_random_substation_change_type(self, candidate_solution):
        chosen_substation = rd.choice(candidate_solution.V_substation)
        chosen_substation_index = next(
            (i for i, substation in enumerate(candidate_solution.V_substation) if substation[0] == chosen_substation[0]), None)
        if chosen_substation_index is not None:
            candidate_solution.V_substation[chosen_substation_index][1] = rd.randint(0, len(self.problem_instance.data["substation_types"])-1)

    def _choose_random_substation_change_cable_type(self, candidate_solution):
        chosen_substation = rd.choice(candidate_solution.V_substation)
        chosen_substation_index = next(
            (i for i, substation in enumerate(candidate_solution.V_substation) if substation[0] == chosen_substation[0]), None)
        if chosen_substation_index is not None:
            candidate_solution.V_substation[chosen_substation_index][2] = rd.randint(0, len(
                self.problem_instance.data["land_substation_cable_types"]) -1)

    def _choose_random_substation_linkage(self, candidate_solution):   #fonction à changer!!
        if len(candidate_solution.V_substation) >= 2:
            chosen_substations = rd.sample(candidate_solution.V_substation, k=2) #2 random elements
            if chosen_substations[0][3] == chosen_substations[1][0] :
                if rd.random() > 0.5:
                    candidate_solution. V_substation=[element for element in candidate_solution.E_ss if
                                                (element[0] != chosen_substations[0][0] and element[0] != chosen_substations[1][0])]
                    candidate_solution.V_substation.append(np.array([chosen_substations[0][0],chosen_substations[0][1],chosen_substations[0][2],None]))

                    candidate_solution.E_ss = [element for element in candidate_solution.E_ss if
                                                (element[0] != chosen_substations[0][0])]


                elif  rd.random() < 0.5:

                    cable_type_index = rd.randint(0, len(self.problem_instance.data["substation_substation_cable_types"])-1)  #it was "land_substation_cable_types"

                    for element in candidate_solution.E_ss:
                      pos1 = chosen_substations[0][0]
                      pos2 = chosen_substations[1][0]
                      # if(self.problem_instance.count==0):
                      #    print("pos1-1 is :",pos1-1)
                      #    print("pos2-1 is :",pos2-1)

                          # Update the cable type in the distance matrix
                      if (element[0] == pos1) or (element[0] == pos2):
                              element[2] = cable_type_index

            elif (chosen_substations[0][3] is not None):

              # Remove elements from candidate_solution.E_ss based on condition
              candidate_solution.E_ss = [element for element in candidate_solution.E_ss if
                                          (element[0] != chosen_substations[0][0])]

              # Choose a new cable_type_index
              cable_type_index = rd.randint(0, len(self.problem_instance.data["substation_substation_cable_types"])-1)


              # Append the new element to candidate_solution.E_ss
              candidate_solution.E_ss.append(
                  [chosen_substations[0][0], chosen_substations[0][3], cable_type_index, 0])
              # if(self.problem_instance.count==0):
              #       print("What we append to E_ss (id1, id2, cable_type, length)  "," chosen_substations[0][0] is " ,chosen_substations[0][0], "chosen_substations[0][3] is ",chosen_substations[0][3])

    def  link_two_substations_with_the_same(self,candidate_solution):
          if len(candidate_solution.V_substation) > 2:
              chosen_indexes = rd.sample(range(len(candidate_solution.V_substation)), k=2) #2 random elements
              chosen_substation_1=candidate_solution.V_substation[chosen_indexes[0]]
              chosen_substation_2=candidate_solution.V_substation[chosen_indexes[1]]
              if chosen_substation_1[3] is not None and chosen_substation_2[3] is not None :

                chosen_substation_1[3]=chosen_substation_2[0]
                chosen_substation_2[3]=chosen_substation_1[0]

                candidate_solution.E_ss = [element for element in candidate_solution.E_ss if
                                          (element[0] != chosen_substation_1[0] and element[0] != chosen_substation_2[0] )]
                cable_type_index = rd.randint(0, len(self.problem_instance.data["substation_substation_cable_types"])-1)



                candidate_solution.E_ss.append(
                  [chosen_substation_1[0], chosen_substation_2[0], cable_type_index, 0])

                candidate_solution.E_ss.append(
                  [chosen_substation_2[0], chosen_substation_1[0], cable_type_index, 0])

    def  change_turbines_linkage(self,candidate_solution):
        # if(self.problem_instance.count==0):
        #     print("Change_turbines_linkage  is here ------------------------------------------------------------------------------------------------------------")

        choose_turbine=rd.choice(candidate_solution.V_turbines)
        station_to_link_with=choose_turbine[1]
        while  station_to_link_with==choose_turbine[1]:
             station_to_link = rd.choice(candidate_solution.V_substation)  #à changer !!
             station_to_link_with=station_to_link[0]

        cable_type_index = rd.randint(0, len(self.problem_instance.data["land_substation_cable_types"])-1)
        station_type_index = rd.randint(0, len(self.problem_instance.data["substation_types"]) -1)

        candidate_solution.V_turbines=[element for element in  candidate_solution.V_turbines if (element[0]!=choose_turbine[0])  ]
        candidate_solution.V_turbines.append(np.array([choose_turbine[0] , station_to_link_with ]))

        t=True
        for turbines in candidate_solution.V_turbines:
                if turbines[1] == choose_turbine[1] :
                    t=False
                    break
        if (t): #it means that no turbines is linked with substation choose_turbine[1]
            index=0
            for i, substation in enumerate(candidate_solution.V_substation):
                if substation[0]==choose_turbine[1]:
                  index=i
                  break


            # if(self.problem_instance.count==0):
            #    print("Change_turbines_linkage condition 2 ( destruction) realised is here ------------------------------------------------------------------------------------------------------------")

            #we'll destroy the substation unfortunately
            new_substations=[element  for element in candidate_solution.V_substation if (element[0]!=index)or (element[3]!=index)]

            for v_sub in candidate_solution.V_substation:
                  if v_sub[3]==index:
                    new_substations.append(np.array([v_sub[0],v_sub[1],v_sub[2],None]))


            candidate_solution.V_substation= new_substations

            #We have to change E_ss too
            candidate_solution.E_ss=[element  for element in candidate_solution.E_ss if (element[0]!=index or element[1]!=index )]




    def _change_substation(self, candidate_solution):
        if len(candidate_solution.V_substation) < self.problem_instance.N_substations:
            # if(self.problem_instance.count==0):
            #   print("Change_substation is here --------------------------------------------------------------------------------------------------------------------------------------------")
            chosen_substation = rd.choice(candidate_solution.V_substation)  # The one to be destroyed
            already_built_indexes = [v_sub[0] for v_sub in candidate_solution.V_substation]

            # Randomly choose an index not in already_built_indexes
            station_to_build_index = None
            while station_to_build_index is None or station_to_build_index +1 in already_built_indexes:
                station_to_build_index = rd.randint(0, len(self.problem_instance.get('substation_locations')) - 1)

            for v_tur in candidate_solution.V_turbines:
                if v_tur[1] == chosen_substation[0]:   #à revoir
                    v_tur[1] = station_to_build_index +1

            # Now I will build the new substation
            candidate_solution.V_substation = [element for element in candidate_solution.V_substation if (element[0] != chosen_substation[0])]
            station_type_index = rd.randint(0, len(self.problem_instance.data["substation_types"]) - 1)
            land_cable_type=rd.randint(0, len(self.problem_instance.data["land_substation_cable_types"]) - 1)

            candidate_solution.V_substation.append(np.array([station_to_build_index+1, station_type_index,land_cable_type, None]))
            #find the previous links
            t=False
            for e in candidate_solution.E_ss:
              if e[0]==chosen_substation[0]:
                previous_link=e
                t=True
                break

            # For linkages
            for e in candidate_solution.E_ss:
              if e[1]== chosen_substation[0]:
                e[1]=station_to_build_index

            candidate_solution.E_ss= [element for element in candidate_solution.E_ss if ((element[0] != chosen_substation[0]) and (element[1] != chosen_substation[0])) ]
            # Choose a new cable_type_index
            cable_type_index = rd.randint(0, len(self.problem_instance.data["substation_substation_cable_types"])-1)

            if t:
              candidate_solution.E_ss.append(np.array([station_to_build_index+1,previous_link[1],cable_type_index,0]))   #possible source of bug


    def build_new_substation(self,candidate_solution):
      if len(candidate_solution.V_substation) < self.problem_instance.N_substations:
            already_built_indexes = [v_sub[0] for v_sub in candidate_solution.V_substation]

            # Randomly choose an index not in already_built_indexes
            station_to_build_index = None
            while station_to_build_index is None or station_to_build_index +1 in already_built_indexes:
                station_to_build_index = rd.randint(0, len(self.problem_instance.get('substation_locations')) - 1)

            count = [0 ] *(self.problem_instance.N_substations+1)
            turbines=[]
            t=True
            for v_r in candidate_solution.V_turbines: #link it with other
                count[v_r[1]]+=1
            for v_r in candidate_solution.V_turbines: #link it with other
                count[v_r[1]]+=1
                if(count[v_r[1]]>=2):
                  t=False   #error if all initialised with one
                  turbines.append(v_r)
            if(t):
              return 0   #Pour sortir
            turbine_linked_with_it=rd.choice(turbines)

            #changing V_turbines

            candidate_solution.V_turbines=[element for element in  candidate_solution.V_turbines if (element[0]!=turbine_linked_with_it[0])  ]
            candidate_solution.V_turbines.append(np.array([turbine_linked_with_it[0] , station_to_build_index+1 ]))



            # Now I will build the new substation
            station_type_index = rd.randint(0, len(self.problem_instance.data["substation_types"]) - 1)
            land_cable_type=rd.randint(0, len(self.problem_instance.data["land_substation_cable_types"]) - 1)

            candidate_solution.V_substation.append(np.array([station_to_build_index+1, station_type_index,land_cable_type, None]))

            return 0






    def add_new_linkage(self, candidate_solution):
      if len(candidate_solution.V_substation) > 2:
            # if(self.problem_instance.count==0):
            #   print("Add new linkage is here ----------------------------------------------------------------------------------------------------------------------------------------------")
            chosen_substations = rd.sample(candidate_solution.V_substation, k=2) #2 random elements
            if chosen_substations[0][3]== None or (chosen_substations[0][3] is not None  and chosen_substations[1][0] != chosen_substations[0][3]):
              # if(self.problem_instance.count==0):
              #   print("Add new linkage condition 2 realised is here --------------------------------------------------------------------------------------------------------------------------------")


              candidate_solution.V_substation = [element for element in candidate_solution.V_substation if (element[0] != chosen_substations[0][0] and element[3] != chosen_substations[1][0])]
              candidate_solution.V_substation.append(np.array([chosen_substations[0][0], chosen_substations[0][1], chosen_substations[0][2], chosen_substations[1][0]]))

              # Remove elements from candidate_solution.E_ss based on condition
              candidate_solution.E_ss = [element for element in candidate_solution.E_ss if (element[0] != chosen_substations[0][0] and element[1]!= chosen_substations[1][0])]
              cable_type_index = rd.randint(0, len(self.problem_instance.data["substation_substation_cable_types"])-1)

              candidate_solution.E_ss.append(np.array([chosen_substations[0][0],chosen_substations[1][0],cable_type_index,0]))
              # if(self.problem_instance.count==0):
              #    print("the size :", len(candidate_solution.E_ss),"-------------------------------------------------------------------------------------------------------------------------------------")


    def choose_candidate(self, candidate_solution):
             # Define your actions
            # Not used , for the moment, we'll do it after
            # actions = [
            #     self.change_turbines_linkage,
            #     self.add_new_linkage,
            #     self._change_substation,
            #     self._choose_random_substation_change_cable_type,
            #     self._choose_random_substation_change_type,
            #     self._choose_random_substation_linkage,
            #     self.build_new_substation,
            # ]


            # if rd.random() < 0.6:
            #     self._choose_random_substation_change_cable_type(candidate_solution)
            # if rd.random() <0.6 :
            #     self._choose_random_substation_change_type(candidate_solution)
            # if    rd.random()<1/3:
            #    self._change_substation(candidate_solution)

            # if rd.random() <0.35:
            #    self.change_turbines_linkage(candidate_solution)
            # if rd.random()<0.3 :
            #   self.build_new_substation(candidate_solution)
            if rd.random()<0.55:
              self. add_new_linkage(candidate_solution)









            # if    rd.random()<1/3:
            #    self._change_substation(candidate_solution)


            # if rd.random() <0.35:
            #    self.change_turbines_linkage(candidate_solution)


            # if rd.random()<0.55:
            #   self. add_new_linkage(candidate_solution)

            # elif rd.random()< 0.4 :
            #    self.link_two_substations_with_the_same(candidate_solution)

            # if rd.random() < 0.6:
            #     self._choose_random_substation_change_cable_type(candidate_solution)
            # if rd.random() <0.6 :
            #     self._choose_random_substation_change_type(candidate_solution)
            # if rd.random() <0.45 :
            #   self._choose_random_substation_linkage(candidate_solution)
            # if rd.random()<0.3 :
            #   self.build_new_substation(candidate_solution)



    def run(self):
        iteration=0
        while self.temperature > 1  :

            candidate_solution = deepcopy(self.current_solution)

            self.choose_candidate(candidate_solution)

            current_cost = self.current_solution.get_cost()
            new_cost = candidate_solution.get_cost()

            best_cost=self.best_solution.get_cost()
            if(iteration%100==0 ):
              print(f'Best Cost: {best_cost:.2e}')
              print(f'Current Cost: {current_cost:.2e}')
              print("tempertaure is :", self.temperature )
            if iteration==0 or iteration ==1:
              print(f'Best Cost {iteration}: {best_cost:.2e}')
              print(f'Current Cost {iteration }: {current_cost:.2e}')


            if self.current_solution.get_cost() < best_cost:
                self.best_solution = deepcopy(self.current_solution)

            ap = self.acceptance_probability(new_cost, current_cost, self.temperature)

            if current_cost > new_cost :
              self.current_solution = candidate_solution

            elif ap > np.random.random():
              self.current_solution = candidate_solution

            if(iteration%self.number_iteration==0):
              self.temperature *= 1 - self.cooling_rate
            # print("new_tempertaure is :", self.temperature )
            iteration+=1

        return self.best_solution


In [ ]:
def find_best_solution(problem_instance, initial_solution, temperature=600, cooling_rate=0.08,number_iteration=220): #5000 0.3 1000
            # temperature = (10**3)/np.log(2)
            sa = SimulatedAnnealing(problem_instance, initial_solution, temperature, cooling_rate,number_iteration)
            best_solution = sa.run()
            return best_solution

In [ ]:
my_problem = problem(input_file)
my_solution = solution(my_problem)
my_solution.random_heuristic_construction()
# my_solution.initialisation_to_compute_the_cost()


print(f'initial solution: {my_solution.get_cost():.2e}')
print("Construction cost ", my_solution.get_construction_cost())
print("operational_cost", my_solution.get_operational_cost())

print("difference ", my_solution.get_cost()-my_solution.get_operational_cost())

initial solution: 1.96e+04
Construction cost  {'substation_substation_cost': 849.6974393004666, 'substation_turbine_cost': 3450.4118654970134, 'substation_cost': 11230.0, 'land_substation_cost': 4118.091411284695}
operational_cost 0.0
difference  19648.200716082174


In [ ]:
print(my_solution.solution_project_format())

{'substations': [{'id': 1, 'land_cable_type': 0, 'substation_type': 19}, {'id': 2, 'land_cable_type': 0, 'substation_type': 19}, {'id': 3, 'land_cable_type': 0, 'substation_type': 10}, {'id': 4, 'land_cable_type': 0, 'substation_type': 17}, {'id': 5, 'land_cable_type': 0, 'substation_type': 1}, {'id': 6, 'land_cable_type': 0, 'substation_type': 18}, {'id': 7, 'land_cable_type': 0, 'substation_type': 15}, {'id': 8, 'land_cable_type': 0, 'substation_type': 7}, {'id': 9, 'land_cable_type': 0, 'substation_type': 16}, {'id': 11, 'land_cable_type': 0, 'substation_type': 17}, {'id': 13, 'land_cable_type': 0, 'substation_type': 16}, {'id': 14, 'land_cable_type': 0, 'substation_type': 12}, {'id': 15, 'land_cable_type': 0, 'substation_type': 12}], 'substation_substation_cables': [{'substation_id': 4, 'other_substation_id': 3, 'cable_type': 16}, {'substation_id': 5, 'other_substation_id': 8, 'cable_type': 3}, {'substation_id': 6, 'other_substation_id': 14, 'cable_type': 13}, {'substation_id': 11,

In [ ]:
best_solution=find_best_solution(my_problem,my_solution)
print(f'best solution: {best_solution.get_cost():.2e}')


Best Cost: 1.96e+04
Current Cost: 1.96e+04
tempertaure is : 600
Best Cost 0: 1.96e+04
Current Cost 0: 1.96e+04
Best Cost 1: 1.96e+04
Current Cost 1: 1.96e+04
Best Cost: 1.96e+04
Current Cost: 2.05e+04
tempertaure is : 552.0
Best Cost: 1.96e+04
Current Cost: 2.08e+04
tempertaure is : 552.0
Best Cost: 1.96e+04
Current Cost: 2.03e+04
tempertaure is : 507.84000000000003
Best Cost: 1.96e+04
Current Cost: 2.18e+04
tempertaure is : 507.84000000000003
Best Cost: 1.96e+04
Current Cost: 2.17e+04
tempertaure is : 467.2128000000001
Best Cost: 1.96e+04
Current Cost: 2.50e+04
tempertaure is : 467.2128000000001
Best Cost: 1.96e+04
Current Cost: 2.50e+04
tempertaure is : 429.83577600000007
Best Cost: 1.96e+04
Current Cost: 2.50e+04
tempertaure is : 429.83577600000007
Best Cost: 1.96e+04
Current Cost: 2.50e+04
tempertaure is : 395.44891392000005
Best Cost: 1.96e+04
Current Cost: 2.50e+04
tempertaure is : 395.44891392000005
Best Cost: 1.96e+04
Current Cost: 2.50e+04
tempertaure is : 395.44891392000005
B

In [ ]:
print(best_solution.get_cost())

6924.50973011594


In [ ]:
print(best_solution.solution_project_format())

{'substations': [{'id': 1, 'land_cable_type': 18, 'substation_type': 18}, {'id': 2, 'land_cable_type': 18, 'substation_type': 18}, {'id': 4, 'land_cable_type': 18, 'substation_type': 18}, {'id': 6, 'land_cable_type': 18, 'substation_type': 12}, {'id': 8, 'land_cable_type': 18, 'substation_type': 18}, {'id': 9, 'land_cable_type': 18, 'substation_type': 18}, {'id': 11, 'land_cable_type': 18, 'substation_type': 18}, {'id': 13, 'land_cable_type': 18, 'substation_type': 18}, {'id': 16, 'land_cable_type': 18, 'substation_type': 18}, {'id': 17, 'land_cable_type': 12, 'substation_type': 18}, {'id': 18, 'land_cable_type': 18, 'substation_type': 18}, {'id': 19, 'land_cable_type': 18, 'substation_type': 18}, {'id': 20, 'land_cable_type': 18, 'substation_type': 18}, {'id': 21, 'land_cable_type': 18, 'substation_type': 19}, {'id': 22, 'land_cable_type': 18, 'substation_type': 18}, {'id': 23, 'land_cable_type': 18, 'substation_type': 18}, {'id': 25, 'land_cable_type': 18, 'substation_type': 18}], 's

In [ ]:
print(best_solution.get_construction_cost())

{'substation_substation_cost': 1381.388082201445, 'substation_turbine_cost': 3149.122029798432, 'substation_cost': 2470.0, 'land_substation_cost': 2711.634119168117}


In [ ]:
print(best_solution.get_cost())

6924.50973011594


In [ ]:
print(best_solution.get_operational_cost())

-2787.634501052054


In [ ]:
from collections import defaultdict

data = best_solution.solution_project_format()

# Create a dictionary to store the occurrences of each ID as a backup station
backup_occurrences = defaultdict(int)

# Iterate through substation_substation_cables
for cable in data['substation_substation_cables']:
    other_substation_id = cable['other_substation_id']
    backup_occurrences[other_substation_id] += 1

# Check if any ID is used more than once as a backup station
duplicated_ids = [id for id, count in backup_occurrences.items() if count > 1]

if duplicated_ids:
    print("IDs used more than once as a backup station:", duplicated_ids)
else:
    print("No IDs are used more than once as a backup station.")


No IDs are used more than once as a backup station.


In [ ]:

best_solution_to_print= deepcopy(best_solution)
best_solution_to_print.normalize_solution()
dict_solution = best_solution_to_print.solution_project_format()

print(dict_solution)

#Code pour adapter le dictionnaire de la solution et le télécharger sous format en json
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

# Spécifiez le chemin du fichier JSON où vous souhaitez sauvegarder le dictionnaire

Nom_fichier_json = 'solution_medium_heuristique.json'

# Apply the custom encoder when writing to JSON
with open(Nom_fichier_json, 'w') as json_file:
    json.dump(dict_solution, json_file, indent=2, cls=NumpyEncoder)

{'substations': [{'id': 3, 'land_cable_type': 1, 'substation_type': 24}, {'id': 5, 'land_cable_type': 1, 'substation_type': 5}, {'id': 6, 'land_cable_type': 1, 'substation_type': 1}, {'id': 7, 'land_cable_type': 1, 'substation_type': 21}, {'id': 9, 'land_cable_type': 1, 'substation_type': 10}, {'id': 11, 'land_cable_type': 1, 'substation_type': 22}, {'id': 13, 'land_cable_type': 1, 'substation_type': 20}, {'id': 14, 'land_cable_type': 1, 'substation_type': 13}, {'id': 15, 'land_cable_type': 1, 'substation_type': 14}, {'id': 17, 'land_cable_type': 1, 'substation_type': 16}, {'id': 23, 'land_cable_type': 1, 'substation_type': 7}, {'id': 24, 'land_cable_type': 1, 'substation_type': 2}, {'id': 26, 'land_cable_type': 1, 'substation_type': 3}, {'id': 27, 'land_cable_type': 1, 'substation_type': 23}, {'id': 29, 'land_cable_type': 1, 'substation_type': 11}], 'substation_substation_cables': [{'substation_id': 6, 'other_substation_id': 15, 'cable_type': 3}, {'substation_id': 7, 'other_substation

In [ ]:
print(my_problem.get("land_substation_cable_types")[0])
print(my_problem.get("substation_substation_cable_types")[0])
print(my_problem.get("wind_turbines")[0])
print(my_problem.get("wind_scenarios")[0])
print(my_problem.get("general_parameters"))
print(my_problem.get("substation_locations")[0])
print(my_problem.get("substation_types")[0])



In [ ]:
prob = 0
for scenario in my_problem.get('wind_scenarios'):
    prob += scenario["probability"]
print(prob)

In [ ]:
print(best_solution.solution_project_format())